## Algorand Contract Deployment Notebook
This notebook contains all code for compilation and deployment of the smart contract on Algorand. <br/>
The source code of the contract is also present here (see 'ASC Source Code' section) <br/><br/>

### Pre-requisites
1. PyTEAL and Jupyter installed (see README.md for instructions)
2. Algod containers running via Docker (see the standard documentation on https://github.com/algorand/sandbox)

### Compilation and deployment Process:
[PyTEAL code] ----> [TEAL code (.teal file)] ----> [deploymentMain()] ----> [TEAL bytecode (.tok file)]  ---> [create_app()] ----> [Deployed Application on Algorand]


In [ ]:
#imports all needed libraries

import base64 

from algosdk.v2client import algod
from algosdk import mnemonic,kmd,wallet
from algosdk import transaction
from algosdk.mnemonic import  to_private_key
import os 

from pyteal import *
from pyteal.ast.bytes import Bytes
from pyteal_helpers import program

from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [ ]:
# # Testnet (donot run this cell when using Local Sandbox)
algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "t5e3i9XJZ95ahVC6OIdlt7orasK0skCP4nYgt44j"
headers = {
   "X-API-Key": algod_token,
}

#Initialize the Algorand SDK client
algod_client = algod.AlgodClient(algod_token, algod_address,headers)

### Get funded accounts
- In case of sandbox, these accounts with private and public keys are automatically generated using the Key Management Daemon (KMD) process running inside the algod docker container

In [ ]:
funded_accounts = [
  {'sk': '+ztKqfdGwqCWitnx+87TBesNQ1OgKso+RA+/Ge2lDtEYTb/7N+CsfaDgzA5Ua8fr1lqNHOTxeBXtoU6eqylGyQ==',
  'pk': 'DBG376ZX4CWH3IHAZQHFI26H5PLFVDI44TYXQFPNUFHJ5KZJI3E2KRCNLQ'},
]
funded_accounts

In [ ]:
# Get account info
algod_client.account_info(funded_accounts[0]['pk'])

## Helper Functions
Contais functions for compiling the TEAL code into bytecode, reading global state of application and creating/calling the application on Algorand network

In [ ]:
# helper function to compile program source
def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

# helper function that formats global state for printing
def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# helper function to read app global state
def read_global_state(client, app_id):
    app = client.application_info(app_id)
    global_state = app['params']['global-state'] if "global-state" in app['params'] else []
    return format_state(global_state)


# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema):
    # define sender as creator
    sender = funded_accounts[0]['pk']

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

    # get node suggested parameters
    params = client.suggested_params()

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(sender, params, on_complete, \
                                            approval_program, clear_program, \
                                            global_schema, local_schema)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 5)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id

# call application
def call_app(client, private_key, index, app_args) :
    # declare sender
    sender = funded_accounts[0]['pk']

    # get node suggested parameters
    params = client.suggested_params()

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])


    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 4)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return
    print("Application called")

### ASC Source code
The below cell needs to be re-executed when the ASC source code is updated.

In [ ]:

def approval_program():
    return Approve()

def clear_state_program():
    return Approve()

### Main Deployment Function
This function takes the TEAL code and generates compiled bytecode and then makes application create transaction



In [ ]:
def compile(approval_filename='./build/approval.teal', clear_state_filename='./build/clear_state.teal'):
    # pyteal to teal 
    compiled1 = ""
    compiled2 = ""
    with open(approval_filename, 'w') as f:
        compiled1 = compileTeal(approval_program(), Mode.Application)
        f.write(compiled1)

    with open(clear_state_filename, 'w') as f:
        compiled2 = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled2)

    # with open(approval_filename+'.tok', 'rb') as f:
    #     approval_bytes = f.read()

    # with open(clear_state_filename+'.tok', 'rb') as f: 
    #     clear_state_bytes = f.read() 

    approval_bytes = compile_program(algod_client,compiled1)
    clear_state_bytes = compile_program(algod_client,compiled2)
        
    return approval_bytes, clear_state_bytes

def deploymentMain(algod_client) :
    # define private keys
    creator_private_key = funded_accounts[0]['sk']

    # declare application state storage (immutable)
    local_ints = 0
    local_bytes = 0
    global_ints = 1
    global_bytes = 1
    global_schema = transaction.StateSchema(global_ints, global_bytes)
    local_schema = transaction.StateSchema(local_ints, local_bytes)

    print("Compiling to TEAL")
    
    # compile program to TEAL assembly
    # with open("./build/approval.teal", "w") as f:
    #     approval_program_teal = program.application(approval_program())
    #     f.write(approval_program_teal)


    # # compile program to TEAL assembly
    # with open("./build/clear.teal", "w") as f:
    #     clear_state_program_teal = program.application(clear_state_program())
    #     f.write(clear_state_program_teal)

    print("Compiled successfully to TEAL")
    print("Compiling to TEAL bytecode (binary)")
    # compile program to binary
    # approval_program_compiled = compile_program(algod_client, approval_program_teal)

    # compile program to binary
    # clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)
    [approval_program_compiled, clear_state_program_compiled] = compile()

    print("Compiled successfully to TEAL bytecode (binary)")
    # print("--------------------------------------------")
    # print("Deploying application......")

    # create new application
    app_id = create_app(algod_client, creator_private_key, approval_program_compiled, clear_state_program_compiled, global_schema, local_schema)
    print("Successfully deployed application with App ID:",app_id)

    # read global state of application
    print("Application Global state:", read_global_state(algod_client, app_id))

    return app_id



#### Compile Only (to .teal)

In [ ]:
[approval_program_compiled, clear_state_program_compiled] = compile()
approval_program_compiled

#### Executing the deployment
The below cell needs to be re-executed if the ASC source code is updated

In [ ]:
# algod_client.account_info(funded_accounts[0]['pk'])
app_id = deploymentMain(algod_client=algod_client)
app_id

In [ ]:
# get deployed app info with app account address
from algosdk.logic import get_application_address

app_id = 95851521  # change this to your app ID
app_info = algod_client.application_info(app_id)
app_info

In [ ]:
"""
    ASC for Algorand-facing endpoint for the custom NFT bridge
    Performs following actions:
    1. Wraps an ERC20 token (like USDC) into a fungible ASA
    2. Adjusts HAND token pool reserves to keep the HAND:stablecoin ratio at 1:1
    3. Fixed price Cross-chain Automated Market Maker(AMM) used:
    HAND Token price = (ERC20 Reserve on Ethereum / HAND token reserve on Algorand) = 1 USD
    4. Swaps Platform token ASAs from one platform to another. Must have the platform token ASAs sent to it and held in reserve
"""

from typing import List
from pyteal import *
from pyteal.ast.bytes import Bytes
from pyteal_helpers import program
 
def approval():
    #globals
    global_owner = Bytes("owner")  # byteslice
    global_reservebalance = Bytes("reservebalance")  # uint64


    #Scratch Varibles for looping over Txn arguments
    scratch_counter_algos = ScratchVar(TealType.uint64)
    scratch_counter_assets = ScratchVar(TealType.uint64)

    #Scratch Varible for Storing Trannsaction argument length
    # asa_arg_len = ScratchVar(TealType.uint64)
    # algo_arg_len = ScratchVar(TealType.uint64)

 
    # operations for adjusting reserves
    op_adjustplus = Bytes("adjustplus")
    op_adjustminus = Bytes("adjustminus")
    op_holdassetsoptin = Bytes("holdassetsoptin")
    op_transferassetout = Bytes("transferassetout")
    op_transferassetprogram = Bytes("transferassetprogram")


    #



    #Operation for Royalty
    op_royalty_algos_transfer= Bytes("payout_algos")
    op_royalty_asset_transfer = Bytes("payout_asset")

 
    adjust_reserve_plus = Seq(
        [
            App.globalPut(global_reservebalance, App.globalGet(global_reservebalance) + Btoi(Txn.application_args[1])),
            Approve(),
        ]
    )
 
    adjust_reserve_minus = Seq(
        [
            App.globalPut(global_reservebalance, App.globalGet(global_reservebalance) - Btoi(Txn.application_args[1])),
            Approve(),
        ]
    )
 
    hold_assets_optin = Seq(
        InnerTxnBuilder.Begin(),
        InnerTxnBuilder.SetFields({
            TxnField.type_enum: TxnType.AssetTransfer,
            # TxnField.asset_receiver: Txn.sender(),
            TxnField.asset_receiver: Global.current_application_address(),
            TxnField.asset_amount: Int(0), #opt-in to the asset
            TxnField.xfer_asset: Txn.assets[0], # Must be in the assets array sent as part of the application call
        }),
        InnerTxnBuilder.Submit(),
        Approve(),
    )
 
    transfer_asset_out = Seq(
        InnerTxnBuilder.Begin(),
        InnerTxnBuilder.SetFields({
            TxnField.type_enum: TxnType.ApplicationCall,
            TxnField.asset_receiver: Txn.sender(),
            TxnField.asset_amount: Btoi(Txn.application_args[1]),  
            TxnField.xfer_asset: Txn.assets[0], # Must be in the assets array sent as part of the application call
        }),
        InnerTxnBuilder.Submit(),
        Approve(),
    )
 
    transfer_asset_programswap = Seq(
        InnerTxnBuilder.Begin(),
        InnerTxnBuilder.SetFields({
            TxnField.type_enum: TxnType.ApplicationCall,
            TxnField.asset_receiver: Txn.application_args[1],
            TxnField.asset_amount: Btoi(Txn.application_args[2]), # this amount is already pre-multiplied with the appropriate multiplier value
            TxnField.xfer_asset: Txn.assets[0], # Must be in the assets array sent as part of the application call
        }),
        InnerTxnBuilder.Submit(),
        Approve(),
    )
 
    payout_royalty_algos = Seq(
        InnerTxnBuilder.Begin(),
        InnerTxnBuilder.SetFields({
            TxnField.type_enum: TxnType.ApplicationCall,
            TxnField.asset_receiver: Txn.application_args[1],
            TxnField.asset_amount: Btoi(Txn.application_args[2]), # this amount is already pre-multiplied with the appropriate multiplier value
            TxnField.xfer_asset: Txn.assets[0], # Must be in the assets array sent as part of the application call
        }),
        InnerTxnBuilder.Submit(),
        Approve(),
    )  
 
    payout_royalty_asa = Seq(
        InnerTxnBuilder.Begin(),
        InnerTxnBuilder.SetFields({
            TxnField.type_enum: TxnType.ApplicationCall,
            TxnField.asset_receiver: Txn.application_args[1],
            TxnField.asset_amount: Btoi(Txn.application_args[2]), # this amount is already pre-multiplied with the appropriate multiplier value
            TxnField.xfer_asset: Txn.assets[0], # Must be in the assets array sent as part of the application call
        }),
        InnerTxnBuilder.Submit(),
        Approve(),
    )
    
    
    #Paying Royalty with multiple arguments

    payout_algos =Seq(
        #InnerTxnBuilder.Begin(),
        #scratch_counter_algos.store(Int(1)),
        #For(counter_algos.store(1), counter_algos.load()<Btoi(Txn.application_args[0])+1,
        For(scratch_counter_algos.store(Int(1)), scratch_counter_algos.load()<Btoi(Txn.application_args[0]),
        scratch_counter_algos.store(scratch_counter_algos.load()+Int(1))
        ).Do(Seq([
            InnerTxnBuilder.Begin(),
            InnerTxnBuilder.SetFields({
                TxnField.type_enum: TxnType.Payment,
                TxnField.receiver: Txn.application_args[scratch_counter_algos.load()],
                TxnField.amount: Btoi(Txn.application_args[scratch_counter_algos.load()+Btoi(Txn.application_args[0])]),
            }),
            InnerTxnBuilder.Submit(),
        ])
        ),
        Approve(),
    )

    payout_asset =Seq(
        For(scratch_counter_assets.store(Int(1)), scratch_counter_assets.load()<Btoi(Txn.application_args[0])+Int(1),
        scratch_counter_assets.store(scratch_counter_assets.load()+Int(1))
        ).Do(Seq([
            InnerTxnBuilder.Begin(),
            InnerTxnBuilder.SetFields({
                TxnField.type_enum: TxnType.AssetTransfer,
                TxnField.asset_receiver: Txn.application_args[scratch_counter_assets.load()],
                TxnField.asset_amount: Btoi(Txn.application_args[scratch_counter_assets.load()+Btoi(Txn.application_args[0])]),
                }),
                 InnerTxnBuilder.Submit(),
            ])
        ),
        Approve(),
    )

    # @Subroutine

    # def SendPayment(Expr:List):

    #     return Seq(
    #         For(scratch_counter_assets.store(Int(1)), scratch_counter_assets.load()<Btoi(Txn.application_args[0])+Int(1),
    #     scratch_counter_assets.store(scratch_counter_assets.load()+Int(1))
    #     ).Do(Seq([
    #         InnerTxnBuilder.Begin(),
    #         InnerTxnBuilder.SetFields({
    #             TxnField.type_enum: TxnType.AssetTransfer,
    #             TxnField.asset_receiver: Txn.application_args[scratch_counter_assets.load()],
    #             TxnField.asset_amount: Btoi(Txn.application_args[scratch_counter_assets.load()+Btoi(Txn.application_args[0])]),
    #             }),
    #              InnerTxnBuilder.Submit(),
    #         ])
    #     ),
    #     Approve(),
    # )

        


 
    return program.event(
        init=Seq(
            [
                App.globalPut(global_owner, Txn.sender()),
                App.globalPut(global_reservebalance, Int(0)),
                Approve(),
            ]
        ),
        no_op=Cond(
            [Txn.application_args[0] == op_adjustplus, adjust_reserve_plus],
            [Txn.application_args[0] == op_adjustminus, adjust_reserve_minus],
            [Txn.application_args[0] == op_holdassetsoptin, hold_assets_optin],
            [Txn.application_args[0] == op_transferassetout, transfer_asset_out],
            [Txn.application_args[0] == op_transferassetprogram, transfer_asset_programswap],

            #
           # [Txn.application_args[0] == op_algos_transfer, ],




            #Operations
            [Txn.application_args[0] == op_royalty_algos_transfer, payout_algos],
            [Txn.application_args[0] == op_royalty_asset_transfer, payout_asset ],
        ),
    )
 
def clear():
    return Int(1)

# M6K33UYTNJDOTH5QPK7N2WOXGR3N7OVQSDPSEVBYNNZGUZLHY2XWSKW3CI